In [18]:
import cv2
import numpy as np
import os
import collections
import mediapipe as mp
import tensorflow as tf
from PIL import ImageFont, ImageDraw, Image

In [19]:
data_path = '../dataset_try'
model_path = './model_tf3.keras'
gesture_labels = sorted(os.listdir(data_path))
sequence_length = 30
zero_landmark = [0.0]*21

In [20]:
model = tf.keras.models.load_model(model_path)
print(f'Loaded model from {model_path}')

Loaded model from ./model_tf3.keras


In [21]:
mean = np.load('mean.npy')   # shape (42,)
std  = np.load('std.npy')    # shape (42,)
# Đảm bảo mean, std đang có dạng 1-chiều length=42
mean = mean.reshape(-1)      # (42,)
std  = std.reshape(-1)       # (42,)

# Tính ZERO_NORMALIZED: nếu không detect thấy tay, ta fill vector (0 − mean)/std
ZERO_NORMALIZED = (np.zeros(42) - mean) / std  # array shape (42,)


In [22]:
# mp_hands = mp.solutions.hands
# hands = mp_hands.Hands(
#     # static_image_mode=False,
#     model_complexity=0,
#     max_num_hands=1,
#     min_detection_confidence=0.5,
#     min_tracking_confidence=0.5
# )
# mp_drawing = mp.solutions.drawing_utils

In [23]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    # static_image_mode=False,
    model_complexity=0,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils
sequence_buffer = collections.deque(maxlen=sequence_length)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Không mở được webcam. Hãy kiểm tra lại.")

print("Đang chạy nhận diện thủ ngữ thời gian thực. Nhấn 'q' để thoát.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Lật ngang để giống gương (tuỳ chọn)
    frame = cv2.flip(frame, 1)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # MediaPipe xử lý
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]

        # Vẽ landmarks lên frame (tuỳ chọn)
        mp_drawing.draw_landmarks(
            frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)
        )

        # Trích 21 điểm (x, y), normalized [0,1]
        landmark_row = []
        for lm in hand_landmarks.landmark:
            landmark_row.append(lm.x)
            landmark_row.append(lm.y)
        # Chuyển thành mảng (42,)
        landmark_row = np.array(landmark_row).reshape(-1)  # (42,)

        # Áp dụng normalization: (x − mean) / std
        landmark_row = (landmark_row - mean) / std         # (42,)
        sequence_buffer.append(landmark_row)

    else:
        # Không phát hiện tay, append giá trị đã normalize của zero keypoints
        sequence_buffer.append(ZERO_NORMALIZED)

    # Khi buffer đạt đủ sequence_length, ta chạy dự đoán
    if len(sequence_buffer) == sequence_length:
        # Chuyển deque thành numpy array shape (sequence_length, 42)
        seq_array = np.array(sequence_buffer)           # (30, 42)
        # Thêm batch dimension: (1, 30, 42)
        input_data = np.expand_dims(seq_array, axis=0)  # (1, 30, 42)

        # Dự đoán
        preds = model.predict(input_data, verbose=0)    # (1, num_classes)
        class_id = np.argmax(preds[0])
        class_name = gesture_labels[class_id]
        confidence = preds[0][class_id]

        # Hiển thị kết quả lên frame
        # text = f'{class_name} ({confidence*100:.1f}%)'
        # cv2.putText(frame, text,
        #             org=(10, 40),
        #             fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        #             fontScale=1.2,
        #             color=(255, 0, 0),
        #             thickness=2)
        text = f'{class_name} ({confidence*100:.1f}%)'
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(frame_rgb)
        draw = ImageDraw.Draw(pil_img)
        # Đường dẫn tới font hỗ trợ tiếng Việt, ví dụ Arial Unicode hoặc Roboto
        font = ImageFont.truetype("arial.ttf", 36)  # Đảm bảo file font này tồn tại

        draw.text((10, 40), text, font=font, fill=(0, 255, 0))

        # Chuyển lại sang BGR cho OpenCV hiển thị
        frame = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

    # Hiển thị frame
    cv2.imshow('Real-Time Sign Recognition', frame)

    # Nhấn 'q' để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# -----------------------------------------------
# 6. Dọn dẹp
# -----------------------------------------------
cap.release()
cv2.destroyAllWindows()
hands.close()

Đang chạy nhận diện thủ ngữ thời gian thực. Nhấn 'q' để thoát.
